In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.model_selection import KFold, cross_val_score

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

#Step 1 reading data

file = r'Heart.csv'
data = pd.read_csv('heart.csv')

dummies = pd.get_dummies(data['famhist'],prefix='famhist', drop_first=False)
data = pd.concat([data,dummies], axis=1)
data = data.drop(['famhist'], axis=1)

inputs=['sbp','tobacco','ldl','adiposity','typea','obesity','alcohol','age']

labels = data['chd']

#scaling whole data set

for each in inputs:
    data[each] = ( data[each] - data[each].min() ) / data[each].max()
    
features = data.drop(['chd'], axis=1)
features.head()

features, labels = np.array(features), np.array(labels)
print(len(features), len(labels))


from sklearn.model_selection import KFold, cross_val_score
nlabels = np.array(labels)
n_epochs=2000
n_hidden1 = 200
n_hidden2 = 50

n_labels= 2
n_features = 10

def model():
    k_fold = KFold(n_splits=30,shuffle=True)
    tf.reset_default_graph()


    
    with tf.name_scope('inputs'):

        inputs = tf.placeholder(tf.float32,[None, 10], name ='inputs' )
        
    with tf.name_scope('target_labels'):
        labels = tf.placeholder(tf.int32, [None,], name='output')
        labels_one_hot = tf.one_hot(labels, 2)
    
    with tf.name_scope('weights'):
        w_h=init_weights([n_features, n_hidden1])
        w_h2=init_weights([n_hidden1, n_hidden2])
        w_o =init_weights([n_hidden2, n_labels])
        
        tf.summary.histogram('hidden_weights1', w_h)
        tf.summary.histogram('hidden_weights2', w_h2)
        tf.summary.histogram('output_weights', w_o)
    
    with tf.name_scope('biases'):
        b1=tf.Variable(tf.zeros([n_hidden1]))
        b2=tf.Variable(tf.zeros([n_hidden2]))
        bo=tf.Variable(tf.zeros(n_labels))
        
        tf.summary.histogram('hidden_biases1', b1)
        tf.summary.histogram('hidden_biases2', b2)
        tf.summary.histogram('output_biases', bo)
    
    
    with tf.name_scope('hidden_layers'):
        inputs=tf.nn.dropout(inputs,0.8)
        hidden_layer1 = tf.nn.bias_add(tf.matmul(inputs,w_h), b1)
        hidden_layer1 = tf.nn.relu(hidden_layer1)
        hidden_layer1=tf.nn.dropout(hidden_layer1,0.5)

        hidden_layer2 = tf.nn.bias_add(tf.matmul(hidden_layer1,w_h2), b2)
        hidden_layer2 = tf.nn.relu(hidden_layer2)
        hidden_layer2=tf.nn.dropout(hidden_layer2,0.5)



        
    with tf.name_scope('predictions'):

        logits = tf.nn.bias_add(tf.matmul(hidden_layer2, w_o), bo, name='logits')
        pred = tf.nn.softmax(logits, name='predictions')
        tf.summary.histogram('predictions', pred)
    
    with tf.name_scope('cost'):
        entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_one_hot, name='cross_entropy')
        cost = tf.reduce_mean(entropy, name='cost')
        tf.summary.scalar('cost', cost)
    
    with tf.name_scope('train'):
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(cost)
        
        
    merged = tf.summary.merge_all()
    
    init = tf.global_variables_initializer()
    accuracy_met=[]
    with tf.Session() as sess:
        
        sess.run(init)
        
        #tensorboard

        train_writer = tf.summary.FileWriter('logs/4', sess.graph)
        for train_indices, test_indices in k_fold.split(features):
            
            train_X = [features[each] for each in train_indices]
            train_X=np.array(train_X)
            train_Y = [nlabels[each] for each in train_indices]
            train_Y=np.array(train_Y)
            test_X = [features[each] for each in test_indices]
            test_Y = [nlabels[each] for each in test_indices]
            test_X=np.array(test_X)
            test_Y=np.array(test_Y)
            
            for i in range(n_epochs):
                summary, _, loss = sess.run([merged,optimizer, cost], feed_dict={inputs:train_X, labels:train_Y})
                train_writer.add_summary(summary, i+1)
                
            # Test model
            correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(labels_one_hot, 1))
            # Calculate accuracy
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            acc=accuracy.eval({inputs: test_X, labels: test_Y})
           
            print("Accuracy:", acc)
            accuracy_met.append(acc)
        
        
        mean=tf.reduce_mean(accuracy_met)
        
        print mean.eval()
        

heart_model = model()      

(462, 462)
('Accuracy:', 0.625)
('Accuracy:', 0.625)
('Accuracy:', 0.75)
('Accuracy:', 0.75)
('Accuracy:', 0.75)
('Accuracy:', 0.8125)
('Accuracy:', 0.875)
('Accuracy:', 0.75)
('Accuracy:', 0.875)
('Accuracy:', 0.875)
('Accuracy:', 1.0)
('Accuracy:', 0.8125)
('Accuracy:', 0.93333334)
('Accuracy:', 0.93333334)
('Accuracy:', 1.0)
('Accuracy:', 0.86666667)
('Accuracy:', 0.86666667)
('Accuracy:', 1.0)
('Accuracy:', 0.86666667)
('Accuracy:', 0.93333334)
('Accuracy:', 0.93333334)
('Accuracy:', 0.86666667)
('Accuracy:', 0.73333335)
('Accuracy:', 0.86666667)
('Accuracy:', 0.86666667)
('Accuracy:', 1.0)
('Accuracy:', 0.93333334)
('Accuracy:', 0.86666667)
('Accuracy:', 0.80000001)
('Accuracy:', 0.86666667)
0.854444
